# Offline preparations for L2 Sea Ice Concentration (SIC) and Sea Ice Edge (SIED) algorithms

**Authors**: T. Lavergne (METNO)

This notebook implements a series of "offline" preparation steps for the Level-2 SIC3H and SIED3H algorithms for the CIMR mission.

Generally, *offline* preparations are steps that are run on a regular basis (e.g. every hours, every day, etc...). They aim at preparing everything needed for the L2 algorithms to run efficiently at once a new l2pp file is available. For example, external Auxiliary Data Files (ADFs) can be fetched and pre-processed on an hourly basis by an *offline* chain, to minimize the waiting time when a new l2pp product arrives.

Specifically, this notebook implements the dynamic tuning of the SIC algorithm against a rolling archive of l2pp files, as is the case in the EUMETSAT OSI SAF and ESA CCI processing chains. In an operational, the rolling archive gives access to typically 5-10 days of L2PP data (full L2PP files or subsets of OZA-compensated TBs and auxiliary fields). In this demonstration prototype, we have to use the only simulated l2pp files we have, which is the SCEPS Polar Scene 1.

In [1]:
import sys, os
from glob import glob
from importlib import reload

# modules in the L2 Sea Ice Concentration ATBD (v2) contain software code that implement the SIC algorithm
# 'local' software modules to implement the SIC algorithm
_syspath = list(sys.path)
for path in _syspath:
    if '/algorithm/' in _syspath:
        break
    else:
        sys.path.insert(0, os.path.abspath('../algorithm'))
        break
from sirrdp import rrdp_file
from pmr_sic import tiepoints as tp
from pmr_sic import hybrid_algo

# modules to read CIMR l2pp files (for the dynamic tuning against an archive of past l2pp files)
# 'l2ppl2 bridge' software modules to read the l2pp files
_syspath = list(sys.path)
for path in _syspath:
    if '/L1BL2Bridge_ATBD/' in _syspath:
        break
    else:
        sys.path.insert(0, os.path.abspath('../../L1BL2Bridge_ATBD/algorithm'))
        break
from tools import io_handler as io
from tools import collocation as coll

from copy import deepcopy
import numpy as np
import json

# encoder class for writing JSON files
class MyEncoder(json.JSONEncoder):
    # https://stackoverflow.com/questions/27050108/convert-numpy-type-to-python/27050186#27050186
    def default(self, obj):
        if isinstance(obj, int):
            return int(obj)
        elif isinstance(obj, float):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

## Locate DI-TDS-JNB and configuration

The input data for this notebook are organized under [DI-TDS-JNB] (Test Data Set for Jupyter NoteBook). The default is to access this TDS online through thredds / opendap unless the environment variable `CIMR_L2PAD_JNB_TDS` is set. If it is set, it is the path to a local copy of the TDS.

We access an L2PP file with all the TBs and auxiliary fields added in the l2pp/L2 Bridge, including OZA compensation terms, as well as atmospheric contribution terms tau, Tup, and Tdown (among others).

In [2]:
try:
    tds_root = os.environ["CIMR_L2PAD_JNB_TDS"]
    print("Use local TDS: {}".format(tds_root))
except KeyError:
    tds_root = 'https://thredds.met.no/thredds/dodsC/cimr/L2PAD/DI-TDS-JNB_draft'
    print("Use online TDS: {}".format(tds_root))


Use local TDS: /home/thomasl/Work/L2PAD/local_TDS/


In [3]:
# This cell has a tag 'parameters' and is used for the CLI with papermill
l2pp_archive_dir = os.path.join(tds_root,'CIMR_L2PP')
out_dir = '../data/output/L2SIC_dynamic_tuning/'
#use_oza_adjust = True

In [4]:
# check the input parameters
if not os.path.isdir(out_dir):
    raise ValueError("The output directory {} does not exist.".format(out_dir))
if not os.path.isdir(l2pp_archive_dir):
    raise ValueError("The L2PP archive directory {} does not exist.".format(l2pp_archive_dir))

Configure the 3 SIC algorithms we will run as part of the L2 SIC + SIED processors.

In [5]:
algos = dict()
algos['KA'] = {'channels':('tb37v', 'tb37h'),  'target_band':'KA'}
algos['CKA'] = {'channels':('tb06v', 'tb37v', 'tb37h'), 'target_band':'C'}
algos['KKA'] = {'channels':('tb19v', 'tb37v', 'tb37h'), 'target_band':'KU'}

tb_dict = {'tb01':'L','tb06':'C','tb10':'X','tb19':'KU','tb37':'KA',}
rev_tb_dict = {v:k for k,v in tb_dict.items()}
bands_needed = []
for alg in algos.keys():
    bands_needed += algos[alg]['channels']
bands_needed = list(set([tb_dict[b[:-1]] for b in bands_needed]))

## An hybrid dynamic / fixed tuning for the SCEPS Polar Scene 1

The SCEPS Polar Scene 1 (v2.1, early 2024) does not reach to high-enough SICs, its peak is around 92%-94% and there are very few values above 95%. This means we cannot extract TB samples for near-100% SIC conditions with lat-lon box. On the other hand, we know the sea-ice emissivity model used by SCEPS for this version of the scene is tuned against the CCI RRDP (AMSR-E and AMSR2).

We thus go for a hybrid approach (dynamic / fixed) for the tuning of our algorithms:
* The OW TB samples are selected from the CIMR l2pp file, in a lat-lon box over open water conditions.
* The CI TB samples are taken from the CCI RRDP (AMSR-E and AMSR2) (pan-Arctic domain)


In [6]:
# locate and read the RRDP files
rrdp_dir = os.path.abspath('../data/CCI_SeaIce_RRDP/')
area = 'nh'
ow_files, ci_files = rrdp_file.find_rrdp_files(rrdp_dir, area=area, years=(2007, 2013, 2018))

channels_needed = []
for alg in algos.keys():
    channels_needed += algos[alg]['channels']
channels_needed = set(channels_needed)

rrdp_pos = rrdp_file.read_RRDP_files(area, ow_files, ci_files, channels=channels_needed)

Number of OW samples:  15954
Number of CI samples:  13441


In [7]:
# boxes of lat/lon: (lat_min, lat_max, lon_min, lon_max)
# To simplify, we use lat/lon boxes here, but in an operational
# mode the region masks would be defined with more advanced geomasks.
OW_box = (73., 76.2, -2., 40)

Locate CIMR L2PP files to use in the training. At this stage, we only have 1 realistic swath, so we must use it. But in an operational context the offline training would access a rolling archive of receent L2PP data, typically 5-10 days.

To simplify, we only extract OW and CI samples from one swath file from the archive.

In [8]:
arch_l2pp_fn_patt = 'L2PAD_l2pp_sceps_geo_polar_scene_1_unfiltered_tot_minimal_nom_nedt_apc_tot_v2p1.nc'
arch_l2pp_fn_patt = os.path.join(l2pp_archive_dir, arch_l2pp_fn_patt)
arch_l2pp_files = glob(arch_l2pp_fn_patt)
if len(arch_l2pp_files) == 0:
    print("WARNING : did not find any good l2pp file in the archive ({})".format(arch_l2pp_fn_patt))

Do the training sample extraction and the storing into tie-point objects for each algorithm in turn (`CKA`, `KKA`, and `KA`).

### SIC algorithm tuning against L2PP TBs.

We perform 2 types of tuning.
1. **OZA-compensated tuning**. We tune the `CKA`, `KKA` and `KA` algorithms on L2PP TBs after OZA-compensation. This is the main tuning, to run the NRT3H and NTC SIC algorithms. During the *online* SIC algorithm, TBs will also be OZA-compensated before entering the algorithms, so we must tune for these TBs here.
2. **Per-feed tuning**. The `KKA` algorithm is tuned per-feed, without prior OZA-compensation of the TBs. In the *online* part of the chain, this tuning will be used in the L1B/L2 Bridge to compute the *fastL2* SICs, before OZA-compensation. See the L1B/L2 Bridge ATBD. The other two algorithms are not tuned per-feed.

### OZA-compensated tuning

In [9]:
reload(tp)
for algo in algos.keys():
    b = algos[algo]['target_band']
    bands_needed = list(set([tb_dict[b[:-1]] for b in algos[algo]['channels']]))
    print(algo, bands_needed, )
    arch_l2pp = io.CIMR_L1B(arch_l2pp_files[0], keep_calibration_view=True, selected_bands=bands_needed)

    # the L2PP files are big and contain many variables we do not need for the offline tuning.
    #    We drop some variables here.
    aux_vars = ('lon', 'lat', 'OZA', 'earth_azimuth',
             'scan_angle', 'horn_scan_angle',)
    l2pp_vars = ('orig_scan', 'orig_sample', 'orig_horn','brightness_temperature_h', 'brightness_temperature_v',
             'oza_compens_RTM_brightness_temperature_v', 'oza_compens_RTM_brightness_temperature_h',)

    keep_vars = tuple(list(aux_vars) + list(l2pp_vars))

    drop_vars = []
    for v in arch_l2pp.data[bands_needed[0]].variables:
        if v not in keep_vars:
            drop_vars.append(v)

    for band in bands_needed:
        arch_l2pp.data[band] = arch_l2pp.data[band].drop_vars(drop_vars, )
    
    # apply the (pre-computed) OZA adjustment fields for all bands.
    arch_l2pp.apply_OZA_adjustment()
    
    # coarsen l2pp samples along the scanlines with a kernel of 5 (horns are *not* combined)
    arch_l2pp = arch_l2pp.coarsen_along_scanlines(kernel=5)
    
    # collocate TBs to the target band of the algorithm.
    if algo == 'KA':
        # we use only one frequency: no need for collocation
        pass
    elif algo == 'KKA':
        # the collocation is done along scanlines (to not mix feeds with different OZAs)
        arch_l2pp = arch_l2pp.collocate_along_scan(target_band=b)
    elif algo == 'CKA':
        # the collocation is done across scanlines and mixes feeds with different OZAs
        #    note : here we also mix forward and aft- scans which is maybe not optimal.
        arch_l2pp = coll.collocate_channels(arch_l2pp.data, b, method='nn')
    else:
        raise NotImplementedError("Collocation step missing for {}".format(algo))
    
    # how to access the lat and lon depends on the collocation type
    if isinstance(arch_l2pp, io.CIMR_L1B):
        _lat = arch_l2pp.data[b].lat
        _lon = arch_l2pp.data[b].lon
    elif str(arch_l2pp['_type']) == 'L1C swath':
        _lat = arch_l2pp['geolocation']['lat']
        _lon = arch_l2pp['geolocation']['lon']
    else:
        raise ValueError("Unrecognized L2PP data type.")
    
    # Create the masks for OW samples from the lat-lon box
    _mask = dict()
    _mask['ow'] = (_lat > OW_box[0])*(_lat < OW_box[1])*(_lon > OW_box[2])*(_lon < OW_box[3]).astype('int')
    
    # Prepare for extracting the training samples. 
    dyn_tbs = dict()
    dyn_tbs['ow'] = dict()
    
    # Extract the brightness temperatures in the OW areas and store in sample dictionaries
    for ch in algos[algo]['channels']:
        # for each input channel to the algorithm (e.g. tb19v), deduce the
        #   name of the variable to be read in the l2pp (or L1C) data structure.
        bn = tb_dict[ch[:-1]]
        pol = ch[-1:]
        
        if isinstance(arch_l2pp, io.CIMR_L1B):
            bnstr = ''
            if bn != b:
                bnstr = '{}_'.format(bn)
            tb_n = '{}brightness_temperature_{}'.format(bnstr, pol)
            # apply the OW selection mask
            w = 'ow'
            dyn_tbs[w][ch] = arch_l2pp.data[b][tb_n].where(_mask[w]).to_masked_array().compressed()
        elif str(arch_l2pp['_type']) == 'L1C swath':
            tb_n = 'brightness_temperature_{}'.format(pol)
            # apply the OW selection mask
            w = 'ow'
            dyn_tbs[w][ch] = arch_l2pp[bn+'_BAND'][tb_n].where(_mask[w]).to_masked_array().compressed()
        else:
            raise ValueError("Unrecognized L2PP data type.")
            
    #
    # OZA-COMPENSATED TUNING, COMBINING ALL FEEDS INTO ONE ALGORITHM
    #
    
    # Transfer the training data in a tie-point object. This involves some processing,
    #  like computing the tie-points and their covariance matrices. 
    ow_tp = tp.OWTiepoint(source='CIMRl2pp-ALLFEEDS', tbs=dyn_tbs['ow'])
    ci_tp = tp.CICETiepoint(source='CCI-RRDP', tbs=rrdp_pos['ci'])
    ow_tp.instr = ci_tp.instr = 'CIMR'
    print("OW tie-point for {} ALLFEEDS: {}".format(algo, ow_tp.tp))

    # Tune the SIC algorithm on these tie-points
    tuned_algo = hybrid_algo.HybridSICAlgo(algos[algo]['channels'], ow_tp, ci_tp)

    # Store the algorithm on disk (JSON file) for later re-use
    json_fn = os.path.join(out_dir,'{}_sic_{}.json'.format(algo.upper(), ow_tp.source.upper()))
    with open(json_fn, 'w') as fp_out:
        json.dump(tuned_algo.strip().to_dict(), fp_out, indent=4, sort_keys=True, cls=MyEncoder)

    print("{} SIC configuration file is in {}".format(algo, json_fn))

KA ['KA']
OW tie-point for KA ALLFEEDS: [139.44649376 210.87917447]
KA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KA_sic_CIMRL2PP-ALLFEEDS.json
CKA ['KA', 'C']
OW tie-point for CKA ALLFEEDS: [157.31183573 139.4479275  210.87598581]
CKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/CKA_sic_CIMRL2PP-ALLFEEDS.json
KKA ['KA', 'KU']
INFO: Collocate KA -> KU along scan
OW tie-point for KKA ALLFEEDS: [182.36086616 139.4382549  210.87589254]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-ALLFEEDS.json


### Per-feed tuning

In [10]:
for algo in ('KKA',):
    
    b = algos[algo]['target_band']
    bands_needed = list(set([tb_dict[b[:-1]] for b in algos[algo]['channels']]))
    print(algo, bands_needed, )

    arch_l2pp = io.CIMR_L1B(arch_l2pp_files[0], keep_calibration_view=True, selected_bands=bands_needed)

    # the L2PP files are big and contain many variables we do not need for the offline tuning.
    #    We drop some variables here.
    aux_vars = ('lon', 'lat', 'OZA', 'earth_azimuth',
             'scan_angle', 'horn_scan_angle',)
    l2pp_vars = ('orig_scan', 'orig_sample', 'orig_horn','brightness_temperature_h', 'brightness_temperature_v',)

    keep_vars = tuple(list(aux_vars) + list(l2pp_vars))

    drop_vars = []
    for v in arch_l2pp.data[bands_needed[0]].variables:
        if v not in keep_vars:
            drop_vars.append(v)

    for band in bands_needed:
        arch_l2pp.data[band] = arch_l2pp.data[band].drop_vars(drop_vars, )
    
    # coarsen l2pp samples along the scanlines with a kernel of 5 (horns are *not* combined)
    arch_l2pp = arch_l2pp.coarsen_along_scanlines(kernel=5)
    
    # collocate TBs to the target band of the algorithm.
    if algo == 'KA':
        # we use only one frequency: no need for collocation
        pass
    elif algo == 'KKA':
        # the collocation is done along scanlines (to not mix feeds with different OZAs)
        arch_l2pp = arch_l2pp.collocate_along_scan(target_band=b)
    else:
        raise NotImplementedError("Per-feed tuning not implemented (not possible?) for {}".format(algo))
    
    # how to access the lat and lon depends on the collocation type
    if isinstance(arch_l2pp, io.CIMR_L1B):
        _lat = arch_l2pp.data[b].lat
        _lon = arch_l2pp.data[b].lon
    elif str(arch_l2pp['_type']) == 'L1C swath':
        _lat = arch_l2pp['geolocation']['lat']
        _lon = arch_l2pp['geolocation']['lon']
    else:
        raise ValueError("Unrecognized L2PP data type.")
    
    # Create the masks for OW samples from the lat-lon box
    _mask = dict()
    _mask['ow'] = (_lat > OW_box[0])*(_lat < OW_box[1])*(_lon > OW_box[2])*(_lon < OW_box[3]).astype('int')
    
    # Prepare for extracting the training samples. We also store the feed number (e.g. 0-7 for KU/KA)
    #   to be able to later train algorithm for specific feeds.
    dyn_tbs = dict()
    dyn_tbs['ow'] = dict()
    dyn_tbs_feed = dict()
    dyn_tbs_feed['ow'] = None
    
    # Extract the brightness temperatures in the OW areas and store in sample dictionaries
    for ch in algos[algo]['channels']:
        # for each input channel to the algorithm (e.g. tb19v), deduce the
        #   name of the variable to be read in the l2pp (or L1C) data structure.
        bn = tb_dict[ch[:-1]]
        pol = ch[-1:]
        
        if isinstance(arch_l2pp, io.CIMR_L1B):
            bnstr = ''
            if bn != b:
                bnstr = '{}_'.format(bn)
            tb_n = '{}brightness_temperature_{}'.format(bnstr, pol)
            # apply the OW selection mask
            w = 'ow'
            dyn_tbs[w][ch] = arch_l2pp.data[b][tb_n].where(_mask[w]).to_masked_array().compressed()
            # keep the feed number information only for the target band
            if bn == b and dyn_tbs_feed[w] is None:
                dyn_tbs_feed[w] = arch_l2pp.data[b]['orig_horn'].where(_mask[w]).to_masked_array().compressed()
        elif str(arch_l2pp['_type']) == 'L1C swath':
            tb_n = 'brightness_temperature_{}'.format(pol)
            # apply the OW selection mask
            w = 'ow'
            dyn_tbs[w][ch] = arch_l2pp[bn+'_BAND'][tb_n].where(_mask[w]).to_masked_array().compressed()
            # keep the feed number information only for the target band
            if bn == b and dyn_tbs_feed[w] is None:
                dyn_tbs_feed[w] = arch_l2pp[bn+'_BAND']['orig_horn'].where(_mask[w]).to_masked_array().compressed()
        else:
            raise ValueError("Unrecognized L2PP data type.")
    
    #
    # PER-FEED TUNING, ONE FOR EACH FEED, no OZA compensation
    #
    for feed in range(0, io.n_horns[b]):
        FEEDNB = 'FEED{}'.format(feed)
        
        dyn_tbs_f = dict()
        dyn_tbs_f['ow'] = dict()
        
        for ch in algos[algo]['channels']:
            for w in ('ow',):
                dyn_tbs_f[w][ch] = dyn_tbs[w][ch][dyn_tbs_feed[w]==feed]
        
        ow_tp = tp.OWTiepoint(source='CIMRl2pp-{}'.format(FEEDNB), tbs=dyn_tbs_f['ow'])
        ci_tp = tp.CICETiepoint(source='CCI-RRDP', tbs=rrdp_pos['ci'])
        ow_tp.instr = ci_tp.instr = 'CIMR'
        print("OW tie-point for {} {}: {}".format(algo, FEEDNB, ow_tp.tp))
        
        # Tune the SIC algorithm on these tie-points
        tuned_algo = hybrid_algo.HybridSICAlgo(algos[algo]['channels'], ow_tp, ci_tp)

        # Store the algorithm on disk (JSON file) for later re-use
        json_fn = os.path.join(out_dir,'{}_sic_{}.json'.format(algo.upper(), ow_tp.source.upper()))
        with open(json_fn, 'w') as fp_out:
            json.dump(tuned_algo.strip().to_dict(), fp_out, indent=4, sort_keys=True, cls=MyEncoder)

        print("{} SIC configuration file is in {}".format(algo, json_fn))

KKA ['KA', 'KU']
INFO: Collocate KA -> KU along scan
OW tie-point for KKA FEED0: [184.08696665 139.26120481 212.3442934 ]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-FEED0.json
OW tie-point for KKA FEED1: [183.56648083 139.32548574 211.90311752]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-FEED1.json
OW tie-point for KKA FEED2: [183.0547305  139.37276532 211.46769661]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-FEED2.json
OW tie-point for KKA FEED3: [182.54003186 139.40239228 211.03559911]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-FEED3.json
OW tie-point for KKA FEED4: [182.04272814 139.48167391 210.60163844]
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CIMRL2PP-FEED4.json
OW tie-point for KKA FEED5: [181.47872952 139.5071235  210.14570276]
KKA SIC configuration file is in ../data/output/

## SIC tuning against static dataset (ESA CCI RRDP)

At the end, we also run a tuning of each algorithm against the static set of SIC0 and SIC1 data points
from the ESA CCI Round Robin Data Package (AMSR-E + AMSR2 TBs). This would not be used in the operational processing, but is used now to demonstrate the impact of dynamic tuning of the algorithms in the Performance Assessment chapter.

The training data are taken from the ESA CCI Sea Ice Concentration Round Robin Data Package of Pedersen et al. (2019). The relevant data files as well as routines to parse the files are stored in module siddrp/.

> Pedersen, Leif Toudal; Saldo, Roberto; Ivanova, Natalia; Kern, Stefan; Heygster, Georg; Tonboe, Rasmus; et al. (2019): Reference dataset for sea ice concentration. figshare. Dataset. https://doi.org/10.6084/m9.figshare.6626549.v7

In [11]:
# run the tuning for each algorithm in turn
for algo in algos.keys():
    
    # Transfer the training data in a tie-point object. This involves some processing,
    #  like computing the tie-points and their covariance matrices. 
    ow_tp = tp.OWTiepoint(source='CCI-RRDP', tbs=rrdp_pos['ow'])
    ci_tp = tp.CICETiepoint(source='CCI-RRDP', tbs=rrdp_pos['ci'])
    ow_tp.instr = ci_tp.instr = 'AMSRs'
    ow_tp.source = ci_tp.source = 'CCI-RRDP'
    
    # Tune the SIC algorithm on these tie-points
    tuned_algo = hybrid_algo.HybridSICAlgo(algos[algo]['channels'], ow_tp, ci_tp)
    
    # Store the algorithm on disk (JSON file) for later re-use
    json_fn = os.path.join(out_dir,'{}_sic_{}.json'.format(algo.upper(), ow_tp.source.upper()))
    with open(json_fn, 'w') as fp_out:
        json.dump(tuned_algo.strip().to_dict(), fp_out, indent=4, sort_keys=True, cls=MyEncoder)

    print("{} SIC configuration file is in {}".format(algo, json_fn))

KA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KA_sic_CCI-RRDP.json
CKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/CKA_sic_CCI-RRDP.json
KKA SIC configuration file is in ../data/output/L2SIC_dynamic_tuning/KKA_sic_CCI-RRDP.json
